# Convert TIFF to OME-NGFF (alternative)

This notebook only differs from the previous notebook in the __Save as an OME-NGFF__ section.

In [1]:
from tifffile import imread
from os.path import join
import zarr
from ome_zarr.writer import write_image

## Load the original TIFF

In [2]:
img_arr = imread(join('raw_data', 'dog.tiff'))
img_arr.shape

(500, 750, 4)

## Transpose and select channels of interest

In [3]:
# Transpose so that the channel axis is first
img_arr = img_arr.transpose((2, 0, 1))
img_arr.shape

(4, 500, 750)

In [4]:
# Discard the alpha channel (i.e., the A in RGBA)
img_arr = img_arr[0:3, :, :]
img_arr.shape

(3, 500, 750)

## Save as an OME-NGFF

Rather than using the `rgb_img_to_ome_zarr` function from the `vitessce` Python package, you may opt to use `zarr` and `ome_zarr` directly:

In [5]:
z_root = zarr.open_group(join('processed_data', 'dog.ome.zarr'), mode = "w")

default_window = {
  "start": 0,
  "min": 0,
  "max": 255,
  "end": 255
}

write_image(
  image = img_arr,
  group = z_root,
  axes = "cyx",
  omero = {
    "name": "Dog",
    "version": "0.3",
    "rdefs": {},
    "channels": [
      {
        "label": "R",
        "color": "FF0000",
        "window": default_window
      },
      {
        "label": "G",
        "color": "00FF00",
        "window": default_window
      },
      {
        "label": "B",
        "color": "0000FF",
        "window": default_window
      }
    ]
  }
)